## PD_MIT-CS1PD dataset

### Ground truth loading

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nqDataLoader as nq  #data loading library
from sklearn.model_selection import train_test_split
import h5py

### Load all the files

In [2]:
# load ground Ground Truth 
cs1PdFr = pd.read_csv( 'MIT-CS1PD/GT_DataPD_MIT-CS1PD.csv' )
# set Patient ID as index
cs1PdFr = cs1PdFr.set_index('pID')
# load ground Ground Truth 
cs2PdFr = pd.read_csv( 'MIT-CS2PD/GT_DataPD_MIT-CS2PD.csv' )
# set Patient ID as index
cs2PdFr = cs2PdFr.set_index('pID')
# show part of Data Frame
# cs2PdFr.head()

In [3]:
filenames = ['file_1', 'file_2']
patients = pd.DataFrame(columns=['id', 'hold', 'label'])
def load_all(record):
    global filenames, patients
    name = record.name
    
    arr = np.empty(shape=(0, 0))
    arr_press = np.empty(shape=(0, 0))
    arr_release = np.empty(shape=(0, 0))
    keyp = pd.Series()
    for filename in filenames:
        keyPressed, htArr, pressArr, releaseArr = \
                nq.getDataFiltHelper( 'MIT-CS1PD/data_MIT-CS1PD/' + cs1PdFr.loc[name][filename])
        arr = np.append(arr, htArr)
        arr_press = np.append(arr_press, pressArr)
        arr_release = np.append(arr_release, releaseArr)
        keyp = keyp.append(pd.Series(keyPressed))
        
    patients = patients.append({'id': name, 'hold': arr, 'key': keyp, 'label': record['gt'], 'press': arr_press, 
                               'release': arr_release}, ignore_index=True)
    return True

def load_all_d2(record):
    global patients
    filenames = ['file_1']
    name = record.name
    
    arr = np.empty(shape=(0, 0))
    arr_press = np.empty(shape=(0, 0))
    arr_release = np.empty(shape=(0, 0))
    keyp = pd.Series()
    for filename in filenames:
        keyPressed, htArr, pressArr, releaseArr = \
                nq.getDataFiltHelper( 'MIT-CS2PD/data_MIT-CS2PD/' + cs2PdFr.loc[name][filename])
        arr = np.append(arr, htArr)
        arr_press = np.append(arr_press, pressArr)
        arr_release = np.append(arr_release, releaseArr)
        keyp = keyp.append(pd.Series(keyPressed))
        
    patients = patients.append({'id': name, 'hold': arr, 'key': keyp, 'label': record['gt'], 'press': arr_press, 
                               'release': arr_release}, ignore_index=True)
    return True

def remove_quotes(row):    
    return list(row.apply(lambda x: x.replace('"', '')))

    
tmp = cs1PdFr.apply(lambda x: load_all(x), axis=1)
tmp = cs2PdFr.apply(lambda x: load_all_d2(x), axis=1)

patients.set_index('id', inplace=True)

patients['key'] = patients['key'].apply(lambda x: remove_quotes(x))
patients = patients.drop('key', axis=1)

nqDataLoader.py:87: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  data = np.genfromtxt(fileIn, dtype=None, delimiter=',', skip_header=0)


### MLP Encoding

In [4]:
patients.head()

,hold,label,press,release
id,,,,
11,"[0.1713, 0.1432, 0.0655, 0.1188, 0.0737, 0.065...",True,"[2.4568, 2.7748, 2.9139, 3.0805, 3.1564, 3.435...","[2.6281, 2.918, 2.9794, 3.1992, 3.2301, 3.5004..."
60,"[0.0895, 0.0954, 0.1738, 0.1078, 0.166, 0.1361...",False,"[2.6084, 2.8721, 3.9054, 5.2092, 5.893, 6.3525...","[2.6979, 2.9675, 4.0792, 5.317, 6.0589, 6.4886..."
67,"[0.1119, 0.1345, 0.3396, 0.1883, 0.1592, 0.134...",True,"[1.3361, 1.9241, 2.6734, 3.1709, 3.6996, 4.082...","[1.4481, 2.0585, 3.0129, 3.3591, 3.8588, 4.217..."
68,"[0.1194, 0.1337, 0.1847, 0.1499, 0.2137, 0.164...",False,"[1.8314, 2.4218, 2.8949, 3.2413, 3.9616, 6.467...","[1.9508, 2.5555, 3.0796, 3.3912, 4.1754, 6.631..."
70,"[0.1528, 0.208, 0.2084, 0.1539, 0.177, 0.2055,...",True,"[0.6356, 6.0042, 6.4526, 8.0744, 9.0555, 9.484...","[0.7884, 6.2122, 6.661, 8.2283, 9.2325, 9.6897..."


### Shift and subtract

In [20]:
def shift_series(arr):
    series = pd.Series(arr)
    shifted = series.shift()
    shifted[0] = 0
#     shifted = shifted[:-1]
    return np.array(shifted)

patients['shifted_release'] = patients['release'].apply(lambda x: shift_series(x))
print(patients['shifted_release'])


id
11      [0.0, 2.6281, 2.918, 2.9794, 3.1992, 3.2301, 3...
60      [0.0, 2.6979, 2.9675, 4.0792, 5.317, 6.0589, 6...
67      [0.0, 1.4481, 2.0585, 3.0129, 3.3591, 3.8588, ...
68      [0.0, 1.9508, 2.5555, 3.0796, 3.3912, 4.1754, ...
70      [0.0, 0.7884, 6.2122, 6.661, 8.2283, 9.2325, 9...
71      [0.0, 1.435, 1.7675, 2.0797, 2.4775, 2.9592, 3...
72      [0.0, 1.6184, 2.0646, 2.4303, 4.3986, 5.3022, ...
73      [0.0, 0.3343, 0.5247, 0.721, 0.971, 1.2771, 1....
74      [0.0, 0.2271, 0.3739, 0.5179, 0.8719, 1.051, 1...
75      [0.0, 1.4301, 1.4948, 1.6971, 1.9804, 2.1462, ...
76      [0.0, 0.9273, 1.312, 1.4526, 1.5879, 1.7218, 1...
77      [0.0, 0.2085, 1.4196, 1.6244, 1.8538, 2.62, 2....
78      [0.0, 0.3554, 0.8653, 1.1917, 1.6631, 2.0722, ...
79      [0.0, 0.6012, 1.6315, 3.741, 3.9579, 4.4192, 4...
80      [0.0, 3.1508, 3.8503, 4.1828, 4.5389, 5.5123, ...
81      [0.0, 0.6839, 0.9336, 2.3453, 4.0126, 6.9817, ...
82      [0.0, 0.8039, 5.0518, 6.1574, 7.8679, 9.934, 1...
83      [0.

In [22]:
def subtract_series(record):
#     print('Press', record['press'][:10])
#     print('Release', record['release'][:10])
#     print('Shifted rel', record['shifted_release'][:10])
    return np.round(record['press'] - record['shifted_release'], decimals=4)
print(patients['press'])
print(patients['shifted_release'])
patients['between'] = patients.apply(lambda x: subtract_series(x), axis=1)
print(patients['between'])

id
11      [2.4568, 2.7748, 2.9139, 3.0805, 3.1564, 3.435...
60      [2.6084, 2.8721, 3.9054, 5.2092, 5.893, 6.3525...
67      [1.3361, 1.9241, 2.6734, 3.1709, 3.6996, 4.082...
68      [1.8314, 2.4218, 2.8949, 3.2413, 3.9616, 6.467...
70      [0.6356, 6.0042, 6.4526, 8.0744, 9.0555, 9.484...
71      [1.308, 1.5909, 1.8954, 2.3344, 2.8288, 3.1587...
72      [1.4668, 1.8193, 2.2541, 4.2651, 5.08, 5.2977,...
73      [0.2505, 0.4371, 0.6232, 0.9025, 1.1972, 1.458...
74      [0.1715, 0.2981, 0.4474, 0.8055, 0.9505, 1.160...
75      [1.2548, 1.3113, 1.5264, 1.8059, 1.9761, 2.165...
76      [0.7845, 1.0934, 1.2994, 1.4137, 1.569, 1.7172...
77      [0.1508, 1.3536, 1.559, 1.7658, 2.5576, 2.866,...
78      [0.2213, 0.6849, 0.9874, 1.5531, 1.8807, 2.529...
79      [0.4582, 1.5035, 3.6282, 3.8268, 4.3188, 4.600...
80      [3.0166, 3.7245, 4.0889, 4.3821, 5.1571, 6.229...
81      [0.5398, 0.8291, 2.2558, 3.9252, 6.9105, 7.185...
82      [0.6806, 4.9737, 6.0674, 7.7885, 9.8031, 11.20...
83      [0.

In [23]:
# pd.DataFrame(patients.iloc[0][['hold', 'between']]).transpose().apply(pd.Series)
# patients.iloc[0][['hold', 'between']]
tmp = pd.Series()
tmp['hold'] = patients.iloc[0]['hold']
tmp['latency'] = patients.iloc[0]['between']
tmp

hold       [0.1713, 0.1432, 0.0655, 0.1188, 0.0737, 0.065...
latency    [2.4568, 0.1467, -0.0041, 0.1011, -0.0428, 0.2...
dtype: object

In [24]:
X_train = patients[['hold', 'between']]
Y_train = patients['label'].astype(int)

In [25]:
# np.stack(X_train.iloc[0]).shape
X_train = X_train.apply(lambda x: np.stack(x), axis=1)
X_train = pd.DataFrame([X_train, Y_train])
X_train = X_train.transpose()
X_train = X_train.rename(columns={'Unnamed 0': 'data'})

In [26]:
X_train.iloc[0]['data'].shape

(2, 4829)

### Chunk and concat

In [27]:
def chunk_one_patient(series, size, overlap):
#     print(series['data'].shape)
    arr1 = np.array(chunk_array(series['data'][0], size, overlap))
    arr2 = np.array(chunk_array(series['data'][1], size, overlap))
    res = np.concatenate([arr1, arr2], axis=1)
#     print(arr)
    target = np.array([series['label'] for i in range(res.shape[0])])
#     print(target)
    return [res, target]

def chunk_array(array, size, overlap):
#     print(array.shape)
    gen = gen_split_overlap(array, size, overlap)
    
    result = []
    for arr in gen:
        # if it's the last arr add zeros (if it's 0.7 of size)
        if arr.shape[0] != size:
            if np.float(arr.shape[0]) / np.float(size) >= 0.7:
                arr = np.concatenate((arr, np.zeros((size - arr.shape[0], ))))
            else:
                continue
        
        result.append(arr)
#         print(arr.shape)
    return result
    

def gen_split_overlap(seq, size, overlap):
    if size < 1 or overlap < 0:
        raise ValueError('size must be >= 1 and overlap >= 0')

    for i in range(0, len(seq) - overlap, size - overlap):            
        yield seq[i: i + size]

In [28]:
chunk_size = 200  # number of data points in one chunk
overlap = 0.5    # overlapping between chunks from 0 to 1
overlap = int(overlap*chunk_size)

# one_patient = chunk_one_patient(X_train.iloc[0], chunk_size, overlap)
# one_patient = one_patient.transpose()
# one_patient.rename(columns={0: "data", 1: "part", 2: 'row', 3: 'target'})

res_df = pd.DataFrame()
res_df = X_train.apply(lambda x: chunk_one_patient(x, chunk_size, overlap), axis=1)
res_df = pd.DataFrame(res_df.apply(pd.Series))
res_df = res_df.rename(columns={0: "data", 1: "target"})

In [29]:
# X_train.iloc[0]['data'].shape
res_df.iloc[4]['data'].shape

(11, 400)

### Save data

In [14]:
# h5f = h5py.File('train_enc_D.h5', 'w')

In [15]:
# h5f.create_group('train')
# h5f.create_dataset('train', data=X_train)
# h5f.create_dataset('target', data=Y_train)
# h5f.close()

In [30]:
# to_json
res_df.to_json('data_encoding_MLP_chunk{chunk}.json'.format(chunk=chunk_size))